In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Giza Systems

/content/drive/My Drive/Giza Systems


In [ ]:
import pandas as pd

Companies = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ9S7nzJY2JGUNke9B7xZtOyIJDoGXr0uJcRd0G-KPs8yqG_jnCeTg97topq8aw7kit9SoI_CcqfWYv/pub?gid=1459643225&single=true&output=csv')
Companies.head(10)

,Company Name,Apple Developer Name,Android Developer Name
0,Verizon,"""Verizon Wireless"" ;Verizon ; ""Verizon Connect...","""Verizon Consumer Group ;Verizon Connect"""
1,Lyft,"""Lyft, Inc."" ;""Lyft Bikes and Scooters, LLC""","""Lyft, Inc."""
2,Paypal,"""PayPal, Inc.""","""PayPal Mobile"""
3,Tinder,"""Tinder LLC""","""Tinder LLC"""
4,Houseparty,"""Houseparty""","""Houseparty"""
5,9gag,"""9GAG""","""9GAG"""
6,Meta,"""Meta Platforms, Inc.""","""Meta Platforms, Inc."""
7,Google LLC,"""Google""","""Google LLC"""


In [ ]:
# Handle multiple values in the 'Google Play Developer Names' column
Companies['Apple Developer Name'] = Companies['Apple Developer Name'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
Companies['Android Developer Name'] = Companies['Android Developer Name'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
Companies.head(10)

,Company Name,Apple Developer Name,Android Developer Name
0,Verizon,"[""Verizon Wireless"" , Verizon , ""Verizon Conn...","[""Verizon Consumer Group , Verizon Connect""]"
1,Lyft,"[""Lyft, Inc."" , ""Lyft Bikes and Scooters, LLC""]","[""Lyft, Inc.""]"
2,Paypal,"[""PayPal, Inc.""]","[""PayPal Mobile""]"
3,Tinder,"[""Tinder LLC""]","[""Tinder LLC""]"
4,Houseparty,"[""Houseparty""]","[""Houseparty""]"
5,9gag,"[""9GAG""]","[""9GAG""]"
6,Meta,"[""Meta Platforms, Inc.""]","[""Meta Platforms, Inc.""]"
7,Google LLC,"[""Google""]","[""Google LLC""]"


In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import search, app
import pandas as pd
import time


def get_google_play_apps(developername):
    results = search(developername, lang='en', country='us')
    apps_data = []

    for result in results:
        app_id = result['appId']
        app_details = app(app_id)

        developer_name = app_details.get('developer', '')
        if developername.lower() in developer_name.lower():
            app_info = {
                'Company Name': company_name,
                'Application Name': app_details.get('title', 'N/A'),
                'Developer Name': developer_name,
                'Number of Downloads': app_details.get('installs', 'N/A'),
                'Play Store URL': f"https://play.google.com/store/apps/details?id={app_id}",
                'Play Store Reviews Rating': app_details.get('score', 'N/A')
            }
            apps_data.append(app_info)

    return pd.DataFrame(apps_data)
all_apps_data = []

# Iterate over the rows of the CSV and extract developer names
for index, row in Companies.iterrows():
    developers = row['Android Developer Name']
    company_name = row['Company Name']

    for developer_name in developers:
        developer_name = developer_name.strip().replace('"', '')
        print(f"Fetching data for company: {company_name} (Developer: {developer_name})")
        company_apps = get_google_play_apps(developer_name)

        if not company_apps.empty:
            all_apps_data.append(company_apps)

        # Add a delay to avoid rate-limiting
        time.sleep(2)

final_df = pd.concat(all_apps_data, ignore_index=True)
print (final_df)
final_df.to_csv('all_google_play_apps.csv', index=False)
print("Data saved to 'all_google_play_apps.csv'")

Fetching data for company: Verizon (Developer: Verizon Consumer Group)
Fetching data for company: Verizon (Developer: Verizon Connect)
Fetching data for company: Lyft (Developer: Lyft, Inc.)
Fetching data for company: Paypal (Developer: PayPal Mobile)
Fetching data for company: Tinder (Developer: Tinder LLC)
Fetching data for company: Houseparty (Developer: Houseparty)
Fetching data for company: 9gag (Developer: 9GAG)
Fetching data for company: Meta (Developer: Meta Platforms, Inc.)
Fetching data for company: Google LLC (Developer: Google LLC)
   Company Name       Application Name          Developer Name  \
0       Verizon             My Verizon  Verizon Consumer Group   
1       Verizon           Verizon Home  Verizon Consumer Group   
2       Verizon         Verizon Family  Verizon Consumer Group   
3       Verizon        Verizon Protect  Verizon Consumer Group   
4       Verizon          Verizon Cloud  Verizon Consumer Group   
..          ...                    ...                

In [ ]:
import requests
import pandas as pd
import time

def search_apple_apps(developer_name, country='us'):

    """Searches the Apple App Store for apps by a specific developer using the iTunes API."""
    url = f"https://itunes.apple.com/search?term={developer_name}&country={country}&entity=software"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch data for developer: {developer_name}. Status code: {response.status_code}")
        return pd.DataFrame()

    data = response.json()
    apps_data = []

    for app in data.get("results", []):
        fetched_developer_name = app.get("artistName", "N/A")

        # Ensure the fetched app is from the correct developer
        if developer_name.lower() in fetched_developer_name.lower():
            app_info = {
                "Company Name": company_name,
                "Application Name": app.get("trackName", "N/A"),
                "Developer Name": fetched_developer_name,
                "App Store URL": app.get("trackViewUrl", "N/A"),
                "App Store Reviews Rating": app.get("averageUserRating", "N/A"),
                "Number of Downloads": "N/A"  # Apple doesn't provide download data
            }
            apps_data.append(app_info)

    return pd.DataFrame(apps_data)
all_apps_data = []

# Iterate over the rows of the CSV and extract developer names
for index, row in Companies.iterrows():
    developers = row['Apple Developer Name']
    company_name = row['Company Name']

    for developer_name in developers:
        developer_name = developer_name.strip().replace('"', '')  # Clean up any extra spaces
        print(f"Fetching data for company: {company_name} (Developer: {developer_name})")
        company_apps = search_apple_apps(developer_name)

        if not company_apps.empty:
            all_apps_data.append(company_apps)

        # Add a delay to avoid rate-limiting
        time.sleep(2)

if all_apps_data:
    final_df = pd.concat(all_apps_data, ignore_index=True)
    final_df.to_csv("all_apple_app_store_apps.csv", index=False)
    print("Data saved to 'all_apple_app_store_apps.csv'")
else:
    print("No data was fetched for any company.")


Fetching data for company: Verizon (Developer: Verizon Wireless)
Fetching data for company: Verizon (Developer: Verizon)
Fetching data for company: Verizon (Developer: Verizon Connect Inc.)
Fetching data for company: Verizon (Developer: Verizon Sourcing LLC)
Fetching data for company: Lyft (Developer: Lyft, Inc.)
Fetching data for company: Lyft (Developer: Lyft Bikes and Scooters, LLC)
Fetching data for company: Paypal (Developer: PayPal, Inc.)
Fetching data for company: Tinder (Developer: Tinder LLC)
Fetching data for company: Houseparty (Developer: Houseparty)
Fetching data for company: 9gag (Developer: 9GAG)
Fetching data for company: Meta (Developer: Meta Platforms, Inc.)
Fetching data for company: Google LLC (Developer: Google)
Data saved to 'all_apple_app_store_apps.csv'
